In [695]:
import numpy as np
import scipy as sp
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

data_path = '../data/curated/2016_11_sc.parquet'


In [696]:
sub_df = pd.read_parquet(data_path)
sub_df

,tpep_pickup_datetime,tpep_dropoff_datetime,trip_distance,payment_type,fare_amount,tip_amount,total_amount
0,2019-01-01 00:46:40,2019-01-01 00:53:20,1.50,1,7.00,1.65,9.95
1,2019-01-01 00:59:47,2019-01-01 01:18:59,2.60,1,14.00,1.00,16.30
2,2018-12-21 13:48:30,2018-12-21 13:52:40,0.00,1,4.50,0.00,5.80
3,2018-11-28 15:52:25,2018-11-28 15:55:45,0.00,2,3.50,0.00,7.55
4,2018-11-28 15:56:57,2018-11-28 15:58:33,0.00,2,52.00,0.00,55.55
...,...,...,...,...,...,...,...
7696612,2019-01-31 23:37:20,2019-02-01 00:10:43,10.24,0,0.00,0.00,0.00
7696613,2019-01-31 23:28:00,2019-01-31 23:50:50,12.43,0,48.80,0.00,54.60
7696614,2019-01-31 23:11:00,2019-01-31 23:46:00,9.14,0,51.05,0.00,54.60
7696615,2019-01-31 23:03:00,2019-01-31 23:14:00,0.00,0,0.00,9.82,0.00


In [ ]:
sub_df.isnull().sum() 

In [ ]:
### HANDWRITTEN THE STARTING AND ENDING DATETIME
# CHANGE TIME
sub_df = sub_df[(sub_df["tpep_pickup_datetime"] >= "2019-11-01 00:00:00")\
              & (sub_df["tpep_pickup_datetime"] <= "2019-11-30 23:59:59")]

In [ ]:
# clean for trip distance 
sub_df = sub_df[sub_df.trip_distance > 0]
sub_df.shape

In [ ]:
# preprocess for payment type
sub_df["payment_type"].value_counts(ascending=True)

In [ ]:
sub_df = sub_df[(sub_df['payment_type'] == 1) | (sub_df['payment_type'] == 2)]

## plotting payment type distribution
import matplotlib.pyplot as plt
fig, ax = plt.subplots()
ax.hist(sub_df.payment_type, bins=6, range=(1, 7), align='left')
plt.show()
### Since the other three types do not meet 0.5% of population, we can basically ignore these data

In [ ]:
# remove all data that does not follow normal payment trend (i.e. lower than basic fare amount or negative tip amount)
sub_df = sub_df[sub_df.fare_amount >= 2.5]
sub_df = sub_df[sub_df.tip_amount >= 0]
sub_df.shape

In [ ]:
# continuous cleaning by removing all suspicious human error points
# numbers are from 2018.11 cleaning, where I read all extreme large data and distinguish between large number and abnormal data
sub_df = sub_df[(sub_df.trip_distance <= 200) & (sub_df.fare_amount <= 500) & (sub_df.tip_amount <= 230) & (sub_df.total_amount <= 500) ]

In [678]:
sub_df.describe()

,trip_distance,payment_type,fare_amount,tip_amount,total_amount
count,6.928196e+06,6.928196e+06,6.928196e+06,6.928196e+06,6.928196e+06
mean,2.905820e+00,1.254591e+00,1.260958e+01,2.171745e+00,1.844629e+01
std,3.770495e+00,4.356310e-01,1.102401e+01,2.550172e+00,1.361830e+01
min,1.000000e-02,1.000000e+00,2.500000e+00,0.000000e+00,3.300000e+00
25%,9.600000e-01,1.000000e+00,6.500000e+00,0.000000e+00,1.115000e+01
50%,1.600000e+00,1.000000e+00,9.000000e+00,1.890000e+00,1.416000e+01
75%,2.940000e+00,2.000000e+00,1.400000e+01,2.860000e+00,1.980000e+01
max,1.434700e+02,2.000000e+00,4.730000e+02,2.251000e+02,4.868000e+02


In [679]:
# quantile cleaning
cols = ['trip_distance', 'fare_amount', 'tip_amount']
Q0 = sub_df[cols].quantile(0.05)
Q1 = sub_df[cols].quantile(0.25)
Q3 = sub_df[cols].quantile(0.75)
Q4 = sub_df[cols].quantile(0.95)
IQR = Q3 - Q1

In [680]:
# Remove outliers by 1.5IQR
removed_df = sub_df[((sub_df[cols] >= (Q1 - 1.5 * IQR)) & (sub_df[cols] <= (Q3 + 1.5 * IQR))).all(axis=1)]
removed_df.describe()

,trip_distance,payment_type,fare_amount,tip_amount,total_amount
count,6.096158e+06,6.096158e+06,6.096158e+06,6.096158e+06,6.096158e+06
mean,1.753835e+00,1.260340e+00,9.335706e+00,1.706180e+00,1.442563e+01
std,1.174401e+00,4.388202e-01,4.248177e+00,1.405963e+00,5.127106e+00
min,1.000000e-02,1.000000e+00,2.500000e+00,0.000000e+00,3.300000e+00
25%,9.000000e-01,1.000000e+00,6.000000e+00,0.000000e+00,1.070000e+01
50%,1.400000e+00,1.000000e+00,8.500000e+00,1.820000e+00,1.350000e+01
75%,2.300000e+00,2.000000e+00,1.150000e+01,2.650000e+00,1.725000e+01
max,5.910000e+00,2.000000e+00,2.525000e+01,7.150000e+00,7.136000e+01


# boxplots after basic data cleaning
cols = ['trip_distance', 'fare_amount', 'tip_amount','total_amount']
fig, axs = plt.subplots(1, 4, figsize=(15,5))
for i in range(4):
    axs[i].boxplot(removed_df[cols[i]])
    axs[i].set_xlabel(cols[i])
plt.show()

In [681]:
# Filling tips amounts for credit cash payment
# calculate average percentage of tip amount 
card_pmt = removed_df.loc[removed_df['payment_type'] == 1, ['tip_amount','total_amount']]
tip_rate = (card_pmt.tip_amount/card_pmt.total_amount).mean()
tip_rate

0.15001689484311478

In [682]:
removed_df.loc[removed_df['payment_type'] == 2, 'tip_amount'] = removed_df['total_amount'] * tip_rate

/opt/miniconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


In [683]:
removed_df['cashtip'] = removed_df['tip_amount'] * (removed_df['payment_type']-1)
removed_df['total_adjusted_amount'] = removed_df['cashtip']+ removed_df['total_amount']
# removed_df = removed_df.drop(['cashtip'], axis=1)

/opt/miniconda3/lib/python3.8/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


In [684]:
# Calculating income rate as income 
removed_df['income_rate'] = removed_df['total_adjusted_amount'] / removed_df['trip_distance']

In [685]:
removed_df

,tpep_pickup_datetime,tpep_dropoff_datetime,trip_distance,payment_type,fare_amount,tip_amount,total_amount,cashtip,total_adjusted_amount,income_rate
0,2019-02-01 00:59:04,2019-02-01 01:07:27,2.10,1,9.0,2.000000,12.30,0.000000,12.300000,5.857143
3,2019-02-01 00:45:38,2019-02-01 00:51:10,0.80,2,5.5,1.020115,6.80,1.020115,7.820115,9.775144
4,2019-02-01 00:25:30,2019-02-01 00:28:14,0.80,2,5.0,0.945106,6.30,0.945106,7.245106,9.056383
5,2019-02-01 00:38:02,2019-02-01 00:40:57,0.80,2,4.5,0.870098,5.80,0.870098,6.670098,8.337622
6,2019-02-01 00:06:49,2019-02-01 00:10:34,0.90,2,5.0,0.945106,6.30,0.945106,7.245106,8.050118
...,...,...,...,...,...,...,...,...,...,...
7019697,2019-02-28 23:50:34,2019-03-01 00:08:54,3.20,1,14.5,3.660000,21.96,0.000000,21.960000,6.862500
7019698,2019-02-28 23:11:03,2019-02-28 23:17:14,1.22,1,6.5,1.000000,11.30,0.000000,11.300000,9.262295
7019699,2019-02-28 23:18:52,2019-02-28 23:29:29,1.75,1,8.5,1.840000,14.14,0.000000,14.140000,8.080000
7019700,2019-02-28 23:34:45,2019-02-28 23:42:11,1.57,2,7.5,1.695191,11.30,1.695191,12.995191,8.277192


In [686]:
after_prep_df = removed_df[['tpep_pickup_datetime','tpep_dropoff_datetime','total_adjusted_amount','income_rate']]

In [687]:
after_prep_df['tpep_pickup_datetime'].dt.date.unique()

array([datetime.date(2019, 2, 1), datetime.date(2019, 2, 2),
       datetime.date(2019, 2, 3), datetime.date(2019, 2, 4),
       datetime.date(2019, 2, 25), datetime.date(2019, 2, 5),
       datetime.date(2019, 2, 6), datetime.date(2019, 2, 7),
       datetime.date(2019, 2, 8), datetime.date(2019, 2, 28),
       datetime.date(2019, 2, 9), datetime.date(2019, 2, 10),
       datetime.date(2019, 2, 11), datetime.date(2019, 2, 12),
       datetime.date(2019, 2, 13), datetime.date(2019, 2, 14),
       datetime.date(2019, 2, 15), datetime.date(2019, 2, 16),
       datetime.date(2019, 2, 17), datetime.date(2019, 2, 18),
       datetime.date(2019, 2, 19), datetime.date(2019, 2, 20),
       datetime.date(2019, 2, 21), datetime.date(2019, 2, 22),
       datetime.date(2019, 2, 23), datetime.date(2019, 2, 24),
       datetime.date(2019, 2, 26), datetime.date(2019, 2, 27)],
      dtype=object)

In [688]:
df_by_day = after_prep_df.groupby([after_prep_df['tpep_pickup_datetime'].dt.date]).mean()

In [689]:
df_by_day

,total_adjusted_amount,income_rate
tpep_pickup_datetime,,
2019-02-01,12.437314,9.697214
2019-02-02,14.100079,10.657006
2019-02-03,14.322427,10.198611
2019-02-04,14.892689,11.338781
2019-02-05,15.209447,11.500092
2019-02-06,15.281437,11.971606
2019-02-07,15.440677,11.775130
2019-02-08,15.381610,11.971209
2019-02-09,14.974809,11.282439


In [690]:
# first create dataframe by running
# df_by_day.to_csv('../data/curated/'+'cleaned_data')
df_by_day.to_csv('../data/curated/cleaned_data', mode = 'a',header = False)

In [ ]:
# 99% CI
x = removed_df['income_rate']
ir_lower = (np.mean(x)- 2.58*(np.std(x)/np.sqrt(len(x))))
ir_upper = (np.mean(x)+ 2.58*(np.std(x)/np.sqrt(len(x))))                 
                  
print(ir_lower)
print(ir_upper)